# Feature 추가

In [2]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("Top3_Date_Group_data.csv", encoding = 'cp949')
data_trade = pd.read_csv("trade_train.csv")

## 2020-02 ~ 2020-07

In [3]:
data_202002 = data[data['기준년월'] >= 202002]
data_202002.head()

,기준년월,종목번호,매수고객수,그룹번호,종목명,20년7월TOP3대상여부,시장구분,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,거래량,거래금액_만원단위
21,202002,A005930,55,MAD01,삼성전자,Y,코스피,제조업,통신장비 제조업,통신 및 방송 장비 제조업,21157172,112943103
22,202002,A000660,33,MAD01,SK하이닉스,Y,코스피,제조업,전자부품· 컴퓨터· 영상· 음향 및,반도체 제조업,5831965,50901766
23,202002,A005935,23,MAD01,삼성전자우,Y,코스피,제조업,통신장비 제조업,통신 및 방송 장비 제조업,934708,3735606
24,202003,A005930,96,MAD01,삼성전자,Y,코스피,제조업,통신장비 제조업,통신 및 방송 장비 제조업,21157172,112943103
25,202003,A000660,42,MAD01,SK하이닉스,Y,코스피,제조업,전자부품· 컴퓨터· 영상· 음향 및,반도체 제조업,5831965,50901766


### feature1-3 표준산업구분 코드_대분류, 중분류, 소분류

원래있는 column

In [4]:
data_202002[['표준산업구분코드_대분류','표준산업구분코드_중분류','표준산업구분코드_소분류']]

,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류
21,제조업,통신장비 제조업,통신 및 방송 장비 제조업
22,제조업,전자부품· 컴퓨터· 영상· 음향 및,반도체 제조업
23,제조업,통신장비 제조업,통신 및 방송 장비 제조업
24,제조업,통신장비 제조업,통신 및 방송 장비 제조업
25,제조업,전자부품· 컴퓨터· 영상· 음향 및,반도체 제조업
...,...,...,...
1627,정보통신업,정보서비스업,자료처리· 호스팅· 포털 및 기타 인터넷 정보매개 서비스업
1628,제조업,화학물질 및 화학제품 제조업;,기초 화학물질 제조업
1629,정보통신업,정보서비스업,자료처리· 호스팅· 포털 및 기타 인터넷 정보매개 서비스업
1630,정보통신업,정보서비스업,자료처리· 호스팅· 포털 및 기타 인터넷 정보매개 서비스업


### feature2 거래량 -변동성

If 거래량 > 주식발행수 X 3 일때

거래량이 많은 종목으로 인기가 많은 종목이라고 할 수 있으며. 테마주일 확률이 높다.

In [5]:
outstanding_shares = pd.read_csv("outstanding_shares.csv") #주식발행수

In [6]:
outstanding_shares.head()

,Unnamed: 0,종목번호,주식발행수
0,0,A000020,"27,931,470"
1,1,A000080,"70,133,611"
2,2,A000100,"66,856,810"
3,3,A000120,"22,812,344"
4,4,A000140,"23,206,765"


In [7]:
outstanding_shares = outstanding_shares.drop("Unnamed: 0", axis=1) # 필요없는 column제거
outstanding_shares['주식발행수'] = outstanding_shares['주식발행수'].str.replace(",","")
outstanding_shares=outstanding_shares.astype({"주식발행수":'float'})

#### stock 데이터

In [8]:
stock = pd.read_csv("stocks.csv")
stock.head()

,index,기준일자,종목번호,종목명,20년7월TOP3대상여부,시장구분,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,종목시가,종목고가,종목저가,종목종가,거래량,거래금액_만원단위
0,0,20190701,A000020,동화약품,N,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,9680,9840,9680,9710,31341,30581
1,1,20190701,A000080,하이트진로,N,코스피,제조업,음료 제조업,알코올음료 제조업,20550,20550,20200,20300,127186,258535
2,2,20190701,A000100,유한양행,Y,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,259000,267500,250000,251500,395757,10139357
3,3,20190701,A000120,CJ대한통운,N,코스피,운수 및 창고업,육상운송 및 파이프라인 운송업,도로 화물 운송업,136500,137000,133000,136000,62513,846100
4,4,20190701,A000140,하이트진로홀딩스,Y,코스피,금융 및 보험업,금융업,기타 금융업,9060,9190,8910,9000,7661,6896


#### stock과 주식발행수 merge

In [9]:
df = pd.merge(stock,outstanding_shares, on="종목번호", how="left") #stock과 주식발행수 merge
df.head()

,index,기준일자,종목번호,종목명,20년7월TOP3대상여부,시장구분,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,종목시가,종목고가,종목저가,종목종가,거래량,거래금액_만원단위,주식발행수
0,0,20190701,A000020,동화약품,N,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,9680,9840,9680,9710,31341,30581,27931470.0
1,1,20190701,A000080,하이트진로,N,코스피,제조업,음료 제조업,알코올음료 제조업,20550,20550,20200,20300,127186,258535,70133611.0
2,2,20190701,A000100,유한양행,Y,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,259000,267500,250000,251500,395757,10139357,66856810.0
3,3,20190701,A000120,CJ대한통운,N,코스피,운수 및 창고업,육상운송 및 파이프라인 운송업,도로 화물 운송업,136500,137000,133000,136000,62513,846100,22812344.0
4,4,20190701,A000140,하이트진로홀딩스,Y,코스피,금융 및 보험업,금융업,기타 금융업,9060,9190,8910,9000,7661,6896,23206765.0


If 거래량 > 주식발행수 X 3 

True / False

In [10]:
df['거래량_주식발행수'] = df['주식발행수']*30 < df['거래량']
df['거래량_주식발행수']

0         False
1         False
2         False
3         False
4         False
          ...  
286056    False
286057    False
286058    False
286059    False
286060    False
Name: 거래량_주식발행수, Length: 286061, dtype: bool

해당 기준으로 측정했을 때 인기 있는 종목의 개수가 적음

In [11]:
len(df[df['거래량_주식발행수']==True])

5

In [14]:
len(df[(df['기준일자']>20200600) & (df['기준일자']<20200700) & (df['거래량_주식발행수']==True)])

0

### 주식발행수대비거래량비율

In [15]:
df['주식발행수대비거래량비율'] = df['거래량'] / df['주식발행수']  * 100

In [16]:
df.head()

,index,기준일자,종목번호,종목명,20년7월TOP3대상여부,시장구분,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,종목시가,종목고가,종목저가,종목종가,거래량,거래금액_만원단위,주식발행수,거래량_주식발행수,주식발행수대비거래량비율
0,0,20190701,A000020,동화약품,N,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,9680,9840,9680,9710,31341,30581,27931470.0,False,0.112207
1,1,20190701,A000080,하이트진로,N,코스피,제조업,음료 제조업,알코올음료 제조업,20550,20550,20200,20300,127186,258535,70133611.0,False,0.181348
2,2,20190701,A000100,유한양행,Y,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,259000,267500,250000,251500,395757,10139357,66856810.0,False,0.591947
3,3,20190701,A000120,CJ대한통운,N,코스피,운수 및 창고업,육상운송 및 파이프라인 운송업,도로 화물 운송업,136500,137000,133000,136000,62513,846100,22812344.0,False,0.274031
4,4,20190701,A000140,하이트진로홀딩스,Y,코스피,금융 및 보험업,금융업,기타 금융업,9060,9190,8910,9000,7661,6896,23206765.0,False,0.033012


### feature3. New_Feature 주식 상승률

일일주가변동액 = |종목 고가 – 종목저가|/종목시가 * 100

In [ ]:
group = data_trade['그룹번호'].unique()

data_202002['주식상승률'] = 0.0 
df_2 = df_1.iloc[[0]] #pd.DataFrame(columns=[df_1.columns.tolist()])
df_2 = df_2.drop(0,axis=0)  

for i in group:
    
    df_1 = data_202002[data_202002['그룹번호'] == i]
    df_1 = df_1.reset_index(drop=True) 
    
    for j in range(len(df_1)):
        stock_2 = stock[(stock['기준일자']//100 == df_1.loc[j]['기준년월']) & (stock['종목번호'] == df_1.loc[j]['종목번호'])]
        stock_2['일일주식상승률'] = abs(stock_2['종목고가'] - stock_2['종목저가']) / stock_2['종목시가'] * 100
        #df_1['주식상승률'] = 0.0
        a = stock_2['일일주식상승률'].mean()
        df_1['주식상승률'][j] = a
        #df_1.iloc[j]['주식상승률'] = stock_2['일일주식상승률'].mean()
    
    df_2 = pd.concat([df_2,df_1],axis=0,ignore_index=True)
    #df_2 = df_2.append(df_1, ignore_index=True)
    
    print(len(df_2))

In [ ]:
len(df_2)

In [ ]:
len(data_202002)

In [ ]:
df_2

## feature4. 상승, 보합, 하락

# 실제 데이터에 합치기

In [3]:
trade = pd.read_csv("trade_data2.csv")

In [4]:
trade.head()

,Unnamed: 0,Unnamed: 0_x,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값,매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값,매수수량 - 매도수량,Unnamed: 0_y,배당률
0,0,0,201907,MAD01,288,A000660,Y,Y,7,17,19,234,74800,78500,4.946524,-215,13.0,1.23
1,1,1,201907,MAD01,288,A001820,Y,Y,3,4,533,154,47385,46700,-1.445605,379,41.0,0.46
2,2,2,201907,MAD01,288,A004020,Y,Y,3,3,409,528,41411,40000,-3.407307,-119,77.0,3.11
3,3,3,201907,MAD01,288,A005380,Y,Y,4,3,323,451,140602,140628,0.018492,-128,105.0,2.23
4,4,4,201907,MAD01,288,A005930,Y,Y,13,19,34,101,45350,46850,3.307607,-67,114.0,2.43


In [5]:
trade = trade.drop(["Unnamed: 0","Unnamed: 0_x"], axis=1)

In [6]:
stock = pd.read_csv("stocks_end2.csv")

In [7]:
stock.head() 

,Unnamed: 0,index,기준일자,종목번호,종목명,20년7월TOP3대상여부,시장구분,표준산업구분코드_소분류,거래량,주식가격_만원단위,종목고가 - 종목저가/종목시가,주식발행수,시가총액,year,month,day
0,0,0,20190701,A000020,동화약품,N,코스피,의약품 제조업,31341,0.975751,1.652893,27931470.0,2.725415e+07,2019,7,201
1,1,1,20190701,A000080,하이트진로,N,코스피,알코올음료 제조업,127186,2.032732,1.703163,70133611.0,1.425628e+08,2019,7,201
2,2,2,20190701,A000100,유한양행,Y,코스피,의약품 제조업,395757,25.620158,6.756757,66856810.0,1.712882e+09,2019,7,201
3,3,3,20190701,A000120,CJ대한통운,N,코스피,도로 화물 운송업,62513,13.534785,2.930403,22812344.0,3.087602e+08,2019,7,201
4,4,4,20190701,A000140,하이트진로홀딩스,Y,코스피,기타 금융업,7661,0.900144,3.090508,23206765.0,2.088942e+07,2019,7,201


In [8]:
stock = stock.drop(["Unnamed: 0","index"], axis=1)

## 주식가격_만원단위, 거래량, 종목고가 - 종목저가/종목시가	, 시가총액

In [9]:
stock['기준년월'] = stock['기준일자'] // 100
stock['기준년월']

0         201907
1         201907
2         201907
3         201907
4         201907
           ...  
286056    202007
286057    202007
286058    202007
286059    202007
286060    202007
Name: 기준년월, Length: 286061, dtype: int64

In [10]:
df1 = pd.DataFrame(stock.groupby(['기준년월','종목번호'])['거래량','주식가격_만원단위','종목고가 - 종목저가/종목시가','시가총액'].mean()) 
df1

<ipython-input-10-bca98b832c18>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1 = pd.DataFrame(stock.groupby(['기준년월','종목번호'])['거래량','주식가격_만원단위','종목고가 - 종목저가/종목시가','시가총액'].mean())


거래량  주식가격_만원단위  종목고가 - 종목저가/종목시가          시가총액
기준년월   종목번호                                                            
201907 A000020  5.205826e+04   0.948494          2.831054  2.649284e+07
       A000080  3.887390e+05   2.162776          2.576597  1.516833e+08
       A000100  6.275791e+04  23.213805          2.631194  1.552001e+09
       A000120  4.299757e+04  13.540408          2.582613  3.088884e+08
       A000140  1.102576e+06   1.009439          8.235989  2.342580e+07
...                      ...        ...               ...           ...
202007 A336370  5.792286e+05   3.550470          5.079521  1.086080e+08
       A33637K  1.946282e+05   1.871362          8.090833  1.378520e+07
       A33637L  5.336430e+04   3.777018          7.481890  6.217161e+06
       A344820  1.085001e+05   3.310245          3.578296  2.764124e+07
       A353810  1.032519e+05   5.340479          6.664215  1.221222e+07

[13912 rows x 4 columns]

In [11]:
df1 = pd.DataFrame(stock.groupby(['기준년월','종목번호'])['거래량','주식가격_만원단위','종목고가 - 종목저가/종목시가','시가총액'].median()) 
df1

<ipython-input-11-b0a45454de6e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1 = pd.DataFrame(stock.groupby(['기준년월','종목번호'])['거래량','주식가격_만원단위','종목고가 - 종목저가/종목시가','시가총액'].median())


거래량  주식가격_만원단위  종목고가 - 종목저가/종목시가          시가총액
기준년월   종목번호                                                        
201907 A000020   33290.0   0.958951          2.433862  2.678490e+07
       A000080  351356.0   2.180099          2.546296  1.528982e+08
       A000100   41250.0  23.065371          2.227171  1.542077e+09
       A000120   41348.0  13.609790          2.536232  3.104712e+08
       A000140  214267.0   0.988057          5.049505  2.292960e+07
...                  ...        ...               ...           ...
202007 A336370  401513.0   3.550740          4.310141  1.086163e+08
       A33637K   92930.0   1.837838          5.680259  1.353825e+07
       A33637L   22345.0   3.788858          5.257805  6.236650e+06
       A344820   97899.5   3.296841          3.142495  2.752931e+07
       A353810   75930.0   5.285274          5.387475  1.208598e+07

[13912 rows x 4 columns]

In [12]:
final = pd.merge(trade, df1, left_on = ['기준년월','종목번호'], right_index=True)

In [13]:
final.columns

Index(['기준년월', '그룹번호', '그룹내고객수', '종목번호', '그룹내_매수여부', '그룹내_매도여부', '매수고객수',
       '매도고객수', '평균매수수량', '평균매도수량', '매수가격_중앙값', '매도가격_중앙값',
       '매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값', '매수수량 - 매도수량', 'Unnamed: 0_y', '배당률',
       '거래량', '주식가격_만원단위', '종목고가 - 종목저가/종목시가', '시가총액'],
      dtype='object')

In [14]:
final = final.drop(['Unnamed: 0_y','평균매수수량','평균매도수량','매수가격_중앙값', '매도가격_중앙값','그룹내_매수여부', '그룹내_매도여부'],axis=1)

In [15]:
final.corr()['매수고객수'].sort_values()

종목고가 - 종목저가/종목시가               -0.041038
배당률                             0.004024
매수수량 - 매도수량                     0.004528
매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값    0.036572
기준년월                            0.080343
주식가격_만원단위                       0.092710
그룹내고객수                          0.109890
거래량                             0.181309
시가총액                            0.330965
매도고객수                           0.942184
매수고객수                           1.000000
Name: 매수고객수, dtype: float64

In [16]:
group = trade['그룹번호'].unique()

for i in group:
    df = final[final['그룹번호']==i]
    
    print(i)
    print(df.corr()['매수고객수'].sort_values())
    print("\n")

MAD01
종목고가 - 종목저가/종목시가               -0.166307
주식가격_만원단위                      -0.047562
매수수량 - 매도수량                     0.020603
기준년월                            0.091232
매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값    0.091841
배당률                             0.162474
거래량                             0.510085
시가총액                            0.690288
매도고객수                           0.915421
매수고객수                           1.000000
그룹내고객수                               NaN
Name: 매수고객수, dtype: float64


MAD02
주식가격_만원단위                      -0.191849
종목고가 - 종목저가/종목시가               -0.176494
매수수량 - 매도수량                     0.018233
배당률                             0.264915
기준년월                            0.288185
매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값    0.303595
시가총액                            0.505130
거래량                             0.629097
매도고객수                           0.826271
매수고객수                           1.000000
그룹내고객수                               NaN
Name: 매수고객수, dtype: float64


MAD03
배당률                 

MAD18
종목고가 - 종목저가/종목시가               -0.131941
배당률                             0.029030
매수수량 - 매도수량                     0.029405
매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값    0.064852
기준년월                            0.103686
주식가격_만원단위                       0.141690
거래량                             0.340608
시가총액                            0.727951
매도고객수                           0.939313
매수고객수                           1.000000
그룹내고객수                               NaN
Name: 매수고객수, dtype: float64


MAD19
종목고가 - 종목저가/종목시가               -0.159100
주식가격_만원단위                       0.002700
매수수량 - 매도수량                     0.009429
매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값    0.133914
기준년월                            0.148008
배당률                             0.174270
거래량                             0.384055
시가총액                            0.559276
매도고객수                           0.758431
매수고객수                           1.000000
그룹내고객수                               NaN
Name: 매수고객수, dtype: float64


MAD20
종목고가 - 종목저가/종목시가    

배당률                            -0.043059
매수수량 - 매도수량                     0.039208
종목고가 - 종목저가/종목시가                0.053109
매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값    0.135200
기준년월                            0.150482
주식가격_만원단위                       0.173784
거래량                             0.302885
시가총액                            0.452611
매도고객수                           0.908564
매수고객수                           1.000000
그룹내고객수                               NaN
Name: 매수고객수, dtype: float64


MAD38
종목고가 - 종목저가/종목시가               -0.039998
배당률                             0.020090
매수수량 - 매도수량                     0.034417
매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값    0.051926
주식가격_만원단위                       0.110377
기준년월                            0.128330
거래량                             0.299557
시가총액                            0.621045
매도고객수                           0.947861
매수고객수                           1.000000
그룹내고객수                               NaN
Name: 매수고객수, dtype: float64


MAD39
주식가격_만원단위                 

# 금토일월화  수목금토(일)-가능 


### 금 - 정현욱
### 토 - 정현욱 
### 월 - 심소민, 정현욱, 이유진 오늘처럼
